In [ ]:
import os
import sys
import json
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import seaborn as sns

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

sys.path.append("../python/")

from plotutils import *
from metricutils import *

In [ ]:
project_root  = ".."

base_folder = "TE_abs_09"
base_folder = os.path.join(project_root, f"outputs/{base_folder}")

geojson_fname  = "../data/abs_09.geojson"
gdf_patches = gpd.read_file(geojson_fname)
gdf_patches.set_index('id', inplace=True)

prov_names=dict(zip(gdf_patches.index,gdf_patches.name))

In [ ]:
# 1 - Regió sanitària RS11 Alt Pirineu i Aran
# 2 - Regió sanitària RS12 Barcelona
# 3 - Regió sanitària RS13 Camp de Tarragona
# 4 - Regió sanitària RS14 Catalunya Central
# 5 - Regió sanitària RS15 Girona
# 6 - Regió sanitària RS16 Lleida
# 7 - Regió sanitària RS17 Terres de l'Ebre

reg_code = 2

sectores = pd.read_csv("../data/sanitat.csv")
sel_dict = {}
c = 0
sel = []
for index, row in sectores.iterrows():
    if row.Nivell == "Regió sanitària":
        if c > 0:
            sel_dict[c] = sel
        c += 1
        sel = []
    elif row.Nivell == "Àrea bàsica salut":
        cod = row.Codi.zfill(3)
        if cod in gdf_patches.index and cod not in ["381", "379", "377"]:
            sel.append(cod)
    else:
        continue

sel_dict[c] = sel
sel_ids = sel_dict[reg_code]

In [ ]:
best_delta = 7
best_omega = 14
print(best_delta, best_omega)

te_folder = os.path.join(base_folder, f"run_w{best_omega}_d{best_delta}_R_risk_ij/")
Txy = xr.load_dataarray(os.path.join(te_folder, "TE.nc"))
DIxy = calculate_TSxy(Txy)

params_fname = os.path.join(te_folder,"params.json")
with open(params_fname) as fh:
    params = json.load(fh)
omega = params["te_params"]["omega"]
delta = params["te_params"]["delta"]

cases_ds = xr.open_dataset(os.path.join(base_folder, "cases_pop_ds.nc"))
risk_ds = xr.open_dataset(os.path.join(base_folder, "risk_ds.nc"))
cases_var = "new_cases_by_100k"

df_cases = cases_ds[cases_var].to_pandas().unstack()
df_cases.name = cases_var

In [ ]:
Txy_sel = Txy.loc[sel_ids, sel_ids, :]
df_cases_sel = df_cases.loc[df_cases.index.get_level_values(1).isin(sel_ids)]

fig = plot_heatmap_cases(Txy_sel, df_cases_sel, prov_names, sort_values=True, scale_values=True, linewidths=0, cmap="vlag", top_x=50)
plt.subplots_adjust(hspace=0.1, wspace=0.1)
plt.savefig("../figures/supp/FigureS16.tiff", dpi=300)